# Exploration phase — initial testing
Here we try out different models on our data. The comments below explain exactly what is happening.

In [1]:
from kedro.framework.session import KedroSession
from kedro.framework.startup import bootstrap_project
from pathlib import Path
import pandas as pd
from sklearn.base import clone
import time

from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.svm import SVR
from sklearn.ensemble import (
    RandomForestRegressor,
    ExtraTreesRegressor,
    AdaBoostRegressor,
    GradientBoostingRegressor,
)
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

[11/11/25 23:54:46] INFO     Using                                                                  ]8;id=69885;file:///opt/homebrew/Caskroom/miniforge/base/envs/f1-pitstop-advisor/lib/python3.11/site-packages/kedro/framework/project/__init__.py\__init__.py]8;;\:]8;id=899760;file:///opt/homebrew/Caskroom/miniforge/base/envs/f1-pitstop-advisor/lib/python3.11/site-packages/kedro/framework/project/__init__.py#270\270]8;;\
                             '/opt/homebrew/Caskroom/miniforge/base/envs/f1-pitstop-advisor/lib/pyt                
                             hon3.11/site-packages/kedro/framework/project/rich_logging.yml' as                    
                             logging configuration.                                                                

In [2]:
project_path = Path.cwd().parent
bootstrap_project(project_path)
print(f"Ścieżka projektu: {project_path}")

Ścieżka projektu: /Users/kacperziebacz/Desktop/f1-pitstop-advisor-v2


In [3]:
with KedroSession.create(project_path=project_path) as session:
    context = session.load_context()
    dfs = context.catalog.load("circuit_lap_data")

print(f"Załadowano dane dla {len(dfs)} torów:")
for circuit, df in dfs.items():
    print(f"  {circuit}: {df.shape[0]} okrążeń, {df.shape[1]} cech")

[11/11/25 23:54:48] INFO     Kedro is sending anonymous usage data with the sole purpose of improving ]8;id=747685;file:///opt/homebrew/Caskroom/miniforge/base/envs/f1-pitstop-advisor/lib/python3.11/site-packages/kedro_telemetry/plugin.py\plugin.py]8;;\:]8;id=585676;file:///opt/homebrew/Caskroom/miniforge/base/envs/f1-pitstop-advisor/lib/python3.11/site-packages/kedro_telemetry/plugin.py#243\243]8;;\
                             the product. No personal data or IP addresses are stored on our side. To              
                             opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK` environment              
                             variables, or create a `.telemetry` file in the current working                       
                             directory with the contents `consent: false`. To hide this message,                   
                             explicitly grant or deny consent. Read more at                                        
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         

                    INFO     Loading data from circuit_lap_data (PickleDataset)...             ]8;id=289276;file:///opt/homebrew/Caskroom/miniforge/base/envs/f1-pitstop-advisor/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=621019;file:///opt/homebrew/Caskroom/miniforge/base/envs/f1-pitstop-advisor/lib/python3.11/site-packages/kedro/io/data_catalog.py#1046\1046]8;;\

Załadowano dane dla 22 torów:
  Catalunya: 5034 okrążeń, 14 cech
  Spa-Francorchamps: 1592 okrążeń, 14 cech
  Silverstone: 2495 okrążeń, 14 cech
  Singapore: 3778 okrążeń, 14 cech
  Hungaroring: 4951 okrążeń, 14 cech
  Suzuka: 2754 okrążeń, 14 cech
  Paul Ricard: 945 okrążeń, 14 cech
  Austin: 918 okrążeń, 14 cech
  Miami: 2160 okrążeń, 14 cech
  Zandvoort: 5035 okrążeń, 14 cech
  Monte Carlo: 4244 okrążeń, 14 cech
  Montreal: 4307 okrążeń, 14 cech
  Monza: 3898 okrążeń, 14 cech
  Melbourne: 3071 okrążeń, 14 cech
  Spielberg: 1124 okrążeń, 14 cech
  Sakhir: 4415 okrążeń, 14 cech
  Imola: 2442 okrążeń, 14 cech
  Baku: 2734 okrążeń, 14 cech
  Mexico City: 3843 okrążeń, 14 cech
  Jeddah: 3430 okrążeń, 14 cech
  Yas Marina Circuit: 3307 okrążeń, 14 cech
  Las Vegas: 1799 okrążeń, 14 cech


In [4]:
# Prepare regressor configurations for testing

# We test many algorithms with parameter tuning using GridSearchCV.
# The GridSearchCVs here will be used as templates. For each circuit,
# every of the GridSearchCVs below will be cloned and fitted to their data.

# GridSearchCV configurations
model_searches = {
    # Linear regression
    "LinearRegression": GridSearchCV(
        make_pipeline(StandardScaler(), PCA(), LinearRegression()),
        {"pca__n_components": [0.98, 0.95, 0.9]},
    ),
    "RidgeCV": GridSearchCV(
        make_pipeline(StandardScaler(), PCA(), RidgeCV(alphas=(0.1, 1.0, 10.0))),
        {"pca__n_components": [0.98, 0.95, 0.9]},
    ),
    "LassoCV": GridSearchCV(
        make_pipeline(
            StandardScaler(),
            PCA(),
            LassoCV(max_iter=100_000, alphas=[0.001, 0.01, 0.1, 1.0]),
        ),
        {"pca__n_components": [0.98, 0.95, 0.9]},
    ),
    "ElasticNetCV": GridSearchCV(
        make_pipeline(
            StandardScaler(),
            PCA(),
            ElasticNetCV(max_iter=100_000, l1_ratio=[0.2, 0.5, 0.8]),
        ),
        {"pca__n_components": [0.98, 0.95, 0.9]},
    ),
    # Polynomial regression
    "PolynomialLinearRegression": GridSearchCV(
        make_pipeline(
            StandardScaler(), PCA(), PolynomialFeatures(), LinearRegression()
        ),
        {"polynomialfeatures__degree": [2, 3], "pca__n_components": [0.98, 0.95, 0.9]},
    ),
    "PolynomialRidgeCV": GridSearchCV(
        make_pipeline(
            StandardScaler(),
            PCA(),
            PolynomialFeatures(),
            RidgeCV(alphas=(0.1, 1.0, 10.0)),
        ),
        {"polynomialfeatures__degree": [2, 3], "pca__n_components": [0.98, 0.95, 0.9]},
    ),
    "PolynomialLassoCV": GridSearchCV(
        make_pipeline(
            StandardScaler(),
            PCA(),
            PolynomialFeatures(),
            LassoCV(max_iter=100_000, alphas=[0.001, 0.01, 0.1]),
        ),
        {"polynomialfeatures__degree": [2, 3], "pca__n_components": [0.98, 0.95, 0.9]},
    ),
    "PolynomialElasticNetCV": GridSearchCV(
        make_pipeline(
            StandardScaler(),
            PCA(),
            PolynomialFeatures(),
            ElasticNetCV(max_iter=100_000, l1_ratio=[0.2, 0.5, 0.8]),
        ),
        {"polynomialfeatures__degree": [2, 3], "pca__n_components": [0.98, 0.95, 0.9]},
    ),
    # Bagging models
    "RandomForestRegressor": GridSearchCV(
        RandomForestRegressor(random_state=42, n_jobs=-1),
        {
            "n_estimators": [100, 200, 400],
            "max_depth": [5, 10, 20, None],
            "min_samples_split": [2, 5, 10],
        },
    ),
    "ExtraTreesRegressor": GridSearchCV(
        ExtraTreesRegressor(random_state=42, n_jobs=-1),
        {
            "n_estimators": [100, 200, 400],
            "max_depth": [5, 10, 20, None],
            "min_samples_split": [2, 5, 10],
        },
    ),
    # Boosting models
    "AdaBoostRegressor": GridSearchCV(
        AdaBoostRegressor(random_state=42),
        {"n_estimators": [50, 100, 200], "learning_rate": [0.01, 0.1, 0.5, 1.0]},
    ),
    "GradientBoostingRegressor": GridSearchCV(
        GradientBoostingRegressor(random_state=42),
        {
            "n_estimators": [100, 200],
            "learning_rate": [0.01, 0.05, 0.1],
            "max_depth": [3, 5],
            "subsample": [0.8, 1.0],
        },
    ),
    "XGBRegressor": GridSearchCV(
        XGBRegressor(
            random_state=42, n_jobs=-1, objective="reg:squarederror", verbosity=0
        ),
        {
            "n_estimators": [100, 200, 400],
            "max_depth": [3, 6, 10],
            "learning_rate": [0.01, 0.1, 0.3],
            "subsample": [0.8, 1.0],
            "colsample_bytree": [0.8, 1.0],
        },
    ),
    # Support vector models
    "SVR_linear": GridSearchCV(
        make_pipeline(StandardScaler(), PCA(), SVR(kernel="linear")),
        {"svr__C": [0.1, 1, 10, 100], "pca__n_components": [0.98, 0.95, 0.9]},
    ),
    "SVR_rbf": GridSearchCV(
        make_pipeline(StandardScaler(), PCA(), SVR(kernel="rbf")),
        {
            "svr__C": [0.1, 1, 10],
            "svr__gamma": ["scale", 0.01, 0.1, 1.0],
            "pca__n_components": [0.98, 0.95, 0.9],
        },
    ),
    # MLP
    "MLPRegressor": GridSearchCV(
        make_pipeline(
            StandardScaler(), PCA(), MLPRegressor(max_iter=100_000, random_state=42)
        ),
        {
            "mlpregressor__hidden_layer_sizes": [
                (16,),
                (24,),
                (24, 12),
                (16, 16),
                (16, 8),
            ],
            "mlpregressor__activation": ["relu", "tanh"],
            "mlpregressor__alpha": [0.0001, 0.001, 0.01],
            "mlpregressor__learning_rate_init": [0.001, 0.01],
            "pca__n_components": [0.98, 0.95, 0.9],
        },
    ),
}

In [5]:
# Fit every single circuit/GridSearch configuration
models_and_circuits = {}

for name in model_searches.keys():
    models_and_circuits[name] = {}

for circuit, data in dfs.items():
    print(f"Fitting models for {circuit}")
    circuit_start = time.time()

    X = data.drop(["LapTimeZScore", "Compound"], axis=1).astype(float)
    y = data["LapTimeZScore"].astype(float)

    print(f"  Shape: {X.shape}, Kolumny: {list(X.columns)}")

    X = X.dropna()
    y = y.loc[X.index]

    # TODO: wersja do spróbowania - zamiast usuwania NAN SimpleImputer z Sklearn
    # mask = ~y.isna()
    # X = X[mask]
    # y = y[mask]
    #
    # imputer = SimpleImputer(strategy='most_frequent')
    # X_imputed = imputer.fit_transform(X)
    # X = pd.DataFrame(X_imputed, columns=X.columns, index=X.index)

    print(f"  Shape po imputajci/usunięciu NAN: {X.shape}, Kolumny: {list(X.columns)}")

    for name, model_search in model_searches.items():
        print(f"Fitting {name};".ljust(50), end="")
        model_start = time.time()

        model_search_copy = clone(model_search)
        model_search_copy.fit(X, y)
        models_and_circuits[name][circuit] = model_search_copy

        print(f"took {round(time.time() - model_start, 2)} seconds")

    print(
        f'Took a total of {round(time.time() - circuit_start, 2)} seconds to fit all models for circuit "{circuit}"\n'
    )

Fitting models for Catalunya
Fitting LinearRegression;                         took 0.05 seconds
Fitting RidgeCV;                                  took 0.05 seconds
Fitting LassoCV;                                  took 0.07 seconds
Fitting ElasticNetCV;                             took 0.55 seconds
Fitting PolynomialLinearRegression;               took 0.47 seconds
Fitting PolynomialRidgeCV;                        took 0.92 seconds
Fitting PolynomialLassoCV;                        took 4.24 seconds
Fitting PolynomialElasticNetCV;                   took 19.03 seconds
Fitting RandomForestRegressor;                    took 39.77 seconds
Fitting ExtraTreesRegressor;                      took 23.29 seconds
Fitting AdaBoostRegressor;                        took 10.32 seconds
Fitting GradientBoostingRegressor;                took 43.31 seconds
Fitting XGBRegressor;                             took 159.3 seconds
Fitting SVR_linear;                               took 333.76 seconds
Fitting SVR

[11/12/25 00:25:03] WARNING  /opt/homebrew/Caskroom/miniforge/base/envs/f1-pitstop-advisor/lib/pyth ]8;id=958334;file:///opt/homebrew/Caskroom/miniforge/base/envs/f1-pitstop-advisor/lib/python3.11/warnings.py\warnings.py]8;;\:]8;id=234705;file:///opt/homebrew/Caskroom/miniforge/base/envs/f1-pitstop-advisor/lib/python3.11/warnings.py#110\110]8;;\
                             on3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:681:                 
                             ConvergenceWarning: Objective did not converge. You might want to                     
                             increase the number of iterations, check the scale of the features or                 
                             consider increasing regularisation. Duality gap: 1.453e+00, tolerance:                
                             1.760e-01                                                                             
                               model = cd_fast.enet_coordinate_descent_gram(                                       
                                                                                                                   

took 14.8 seconds
Fitting PolynomialElasticNetCV;                   took 36.21 seconds
Fitting RandomForestRegressor;                    took 27.12 seconds
Fitting ExtraTreesRegressor;                      took 17.71 seconds
Fitting AdaBoostRegressor;                        took 5.46 seconds
Fitting GradientBoostingRegressor;                took 20.71 seconds
Fitting XGBRegressor;                             took 146.26 seconds
Fitting SVR_linear;                               took 97.72 seconds
Fitting SVR_rbf;                                  took 12.04 seconds
Fitting MLPRegressor;                             took 357.01 seconds
Took a total of 736.98 seconds to fit all models for circuit "Silverstone"

Fitting models for Singapore
Fitting LinearRegression;                         took 0.04 seconds
Fitting RidgeCV;                                  took 0.04 seconds
Fitting LassoCV;                                  took 0.07 seconds
Fitting ElasticNetCV;                             t

[11/12/25 00:37:22] WARNING  /opt/homebrew/Caskroom/miniforge/base/envs/f1-pitstop-advisor/lib/pyth ]8;id=993515;file:///opt/homebrew/Caskroom/miniforge/base/envs/f1-pitstop-advisor/lib/python3.11/warnings.py\warnings.py]8;;\:]8;id=169859;file:///opt/homebrew/Caskroom/miniforge/base/envs/f1-pitstop-advisor/lib/python3.11/warnings.py#110\110]8;;\
                             on3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:681:                 
                             ConvergenceWarning: Objective did not converge. You might want to                     
                             increase the number of iterations, check the scale of the features or                 
                             consider increasing regularisation. Duality gap: 1.438e+00, tolerance:                
                             2.811e-01                                                                             
                               model = cd_fast.enet_coordinate_descent_gram(                                       
                                                                                                                   

[11/12/25 00:37:23] WARNING  /opt/homebrew/Caskroom/miniforge/base/envs/f1-pitstop-advisor/lib/pyth ]8;id=563446;file:///opt/homebrew/Caskroom/miniforge/base/envs/f1-pitstop-advisor/lib/python3.11/warnings.py\warnings.py]8;;\:]8;id=695436;file:///opt/homebrew/Caskroom/miniforge/base/envs/f1-pitstop-advisor/lib/python3.11/warnings.py#110\110]8;;\
                             on3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:681:                 
                             ConvergenceWarning: Objective did not converge. You might want to                     
                             increase the number of iterations, check the scale of the features or                 
                             consider increasing regularisation. Duality gap: 1.768e+00, tolerance:                
                             2.745e-01                                                                             
                               model = cd_fast.enet_coordinate_descent_gram(                                       
                                                                                                                   

...

                    WARNING  /opt/homebrew/Caskroom/miniforge/base/envs/f1-pitstop-advisor/lib/pyth ]8;id=462078;file:///opt/homebrew/Caskroom/miniforge/base/envs/f1-pitstop-advisor/lib/python3.11/warnings.py\warnings.py]8;;\:]8;id=522509;file:///opt/homebrew/Caskroom/miniforge/base/envs/f1-pitstop-advisor/lib/python3.11/warnings.py#110\110]8;;\
                             on3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:681:                 
                             ConvergenceWarning: Objective did not converge. You might want to                     
                             increase the number of iterations, check the scale of the features or                 
                             consider increasing regularisation. Duality gap: 1.019e+01, tolerance:                
                             2.422e-01                                                                             
                               model = cd_fast.enet_coordinate_descent_gram(                                       
                                                                                                                   

                    WARNING  /opt/homebrew/Caskroom/miniforge/base/envs/f1-pitstop-advisor/lib/pyth ]8;id=360432;file:///opt/homebrew/Caskroom/miniforge/base/envs/f1-pitstop-advisor/lib/python3.11/warnings.py\warnings.py]8;;\:]8;id=810809;file:///opt/homebrew/Caskroom/miniforge/base/envs/f1-pitstop-advisor/lib/python3.11/warnings.py#110\110]8;;\
                             on3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:681:                 
                             ConvergenceWarning: Objective did not converge. You might want to                     
                             increase the number of iterations, check the scale of the features or                 
                             consider increasing regularisation. Duality gap: 3.615e+01, tolerance:                
                             2.290e-01                                                                             
                               model = cd_fast.enet_coordinate_descent_gram(                                       
                                                                                                                   

[11/12/25 04:24:59] WARNING  /opt/homebrew/Caskroom/miniforge/base/envs/f1-pitstop-advisor/lib/pyth ]8;id=836694;file:///opt/homebrew/Caskroom/miniforge/base/envs/f1-pitstop-advisor/lib/python3.11/warnings.py\warnings.py]8;;\:]8;id=145549;file:///opt/homebrew/Caskroom/miniforge/base/envs/f1-pitstop-advisor/lib/python3.11/warnings.py#110\110]8;;\
                             on3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:681:                 
                             ConvergenceWarning: Objective did not converge. You might want to                     
                             increase the number of iterations, check the scale of the features or                 
                             consider increasing regularisation. Duality gap: 2.021e+01, tolerance:                
                             2.234e-01                                                                             
                               model = cd_fast.enet_coordinate_descent_gram(                                       
                                                                                                                   

                    WARNING  /opt/homebrew/Caskroom/miniforge/base/envs/f1-pitstop-advisor/lib/pyth ]8;id=412466;file:///opt/homebrew/Caskroom/miniforge/base/envs/f1-pitstop-advisor/lib/python3.11/warnings.py\warnings.py]8;;\:]8;id=98192;file:///opt/homebrew/Caskroom/miniforge/base/envs/f1-pitstop-advisor/lib/python3.11/warnings.py#110\110]8;;\
                             on3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:681:                 
                             ConvergenceWarning: Objective did not converge. You might want to                     
                             increase the number of iterations, check the scale of the features or                 
                             consider increasing regularisation. Duality gap: 7.337e+01, tolerance:                
                             2.435e-01                                                                             
                               model = cd_fast.enet_coordinate_descent_gram(                                       
                                                                                                                   

took 29.4 seconds
Fitting PolynomialElasticNetCV;                   took 10.68 seconds
Fitting RandomForestRegressor;                    took 29.52 seconds
Fitting ExtraTreesRegressor;                      took 18.87 seconds
Fitting AdaBoostRegressor;                        took 6.53 seconds
Fitting GradientBoostingRegressor;                took 26.41 seconds
Fitting XGBRegressor;                             took 149.21 seconds
Fitting SVR_linear;                               took 161.41 seconds
Fitting SVR_rbf;                                  took 22.05 seconds
Fitting MLPRegressor;                             took 520.58 seconds
Took a total of 976.4 seconds to fit all models for circuit "Jeddah"

Fitting models for Yas Marina Circuit
Fitting LinearRegression;                         took 0.04 seconds
Fitting RidgeCV;                                  took 0.04 seconds
Fitting LassoCV;                                  took 0.07 seconds
Fitting ElasticNetCV;                          

In [6]:
# Save models for later use
context.catalog.save("initial_models", models_and_circuits)
print("Zapisano modele")

[11/12/25 05:04:56] INFO     Kedro is sending anonymous usage data with the sole purpose of improving ]8;id=882275;file:///opt/homebrew/Caskroom/miniforge/base/envs/f1-pitstop-advisor/lib/python3.11/site-packages/kedro_telemetry/plugin.py\plugin.py]8;;\:]8;id=184633;file:///opt/homebrew/Caskroom/miniforge/base/envs/f1-pitstop-advisor/lib/python3.11/site-packages/kedro_telemetry/plugin.py#243\243]8;;\
                             the product. No personal data or IP addresses are stored on our side. To              
                             opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK` environment              
                             variables, or create a `.telemetry` file in the current working                       
                             directory with the contents `consent: false`. To hide this message,                   
                             explicitly grant or deny consent. Read more at                                        
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         

[11/12/25 05:04:57] INFO     Saving data to initial_models (PickleDataset)...                  ]8;id=552187;file:///opt/homebrew/Caskroom/miniforge/base/envs/f1-pitstop-advisor/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=506901;file:///opt/homebrew/Caskroom/miniforge/base/envs/f1-pitstop-advisor/lib/python3.11/site-packages/kedro/io/data_catalog.py#1006\1006]8;;\

Zapisano modele


In [7]:
# Show scores for each GridSearch and circuit
all_scores = {}
for key in models_and_circuits.keys():
    scores = {}
    for circuit, model in models_and_circuits[key].items():
        scores[circuit] = model.best_score_
    all_scores[key] = scores

all_scores = pd.DataFrame(all_scores)

all_scores

,LinearRegression,RidgeCV,LassoCV,ElasticNetCV,PolynomialLinearRegression,PolynomialRidgeCV,PolynomialLassoCV,PolynomialElasticNetCV,RandomForestRegressor,ExtraTreesRegressor,AdaBoostRegressor,GradientBoostingRegressor,XGBRegressor,SVR_linear,SVR_rbf,MLPRegressor
Catalunya,0.082977,0.083098,0.081894,0.081734,0.130970,0.131199,0.155149,0.152717,0.708742,0.676139,0.557348,0.695654,0.724938,-0.013247,0.162125,0.568469
Spa-Francorchamps,0.238238,0.238412,0.238904,0.239064,0.317078,0.322346,0.365884,0.413322,0.915839,0.908303,0.863359,0.911897,0.916715,0.146229,0.444979,0.863216
Silverstone,0.153171,0.153314,0.139876,0.144137,0.168233,0.171902,0.257637,0.273600,0.694162,0.742581,0.466580,0.767565,0.800788,-0.040327,0.414570,0.538791
Singapore,0.075462,0.075778,0.074477,0.088084,0.079758,0.123315,0.212481,0.119479,0.536864,0.606373,0.495670,0.638594,0.741458,0.070110,0.244049,0.450528
Hungaroring,0.077608,0.077608,0.076575,0.077565,0.088259,0.089409,0.118067,0.112234,0.511993,0.680431,0.602407,0.551330,0.684105,0.015998,0.059768,0.623576
Suzuka,0.194786,0.194942,0.195465,0.196563,0.306188,0.298366,0.305931,0.303483,0.766678,0.819790,0.737291,0.827299,0.826470,0.106739,0.342851,0.652428
Paul Ricard,0.304038,0.304827,0.304221,0.303474,0.251547,0.520739,0.496810,0.560459,0.867341,0.885790,0.818549,0.887277,0.896352,0.210763,0.765324,0.841277
Austin,0.112689,0.114931,0.114427,0.115651,0.102327,0.263041,0.312757,0.305344,0.891936,0.875339,0.854516,0.914460,0.921024,-0.052613,0.624162,0.674486
Miami,0.161720,0.161694,0.161828,0.162218,0.265205,0.267307,0.317009,0.312050,0.793402,0.840128,0.653587,0.806499,0.827863,0.040956,0.414464,0.569850
Zandvoort,0.068847,0.068952,0.069654,0.069876,0.208122,0.208045,0.234060,0.270772,0.824571,0.820640,0.357220,0.863794,0.866933,-0.062803,0.421001,0.556423


In [8]:
# Show score statistics for each model
# MinScore is very important. A good model should perform reasonably well for all tracks.
model_scores_df = pd.DataFrame(
    {
        "MeanScore": all_scores.mean(axis="index"),
        "MedianScore": all_scores.median(axis="index"),
        "ScoreVariance": all_scores.var(axis="index"),
        "MinScore": all_scores.min(axis="index"),
    }
)

model_scores_df.sort_values(by=["MeanScore"], ascending=False)

,MeanScore,MedianScore,ScoreVariance,MinScore
XGBRegressor,0.814173,0.834430,0.006970,0.663191
GradientBoostingRegressor,0.788184,0.826205,0.012961,0.551330
ExtraTreesRegressor,0.780996,0.815815,0.009021,0.606373
RandomForestRegressor,0.770048,0.805139,0.013212,0.511993
MLPRegressor,0.611230,0.585080,0.017165,0.429922
AdaBoostRegressor,0.589864,0.577167,0.029688,0.357220
SVR_rbf,0.399338,0.414517,0.031613,0.059768
PolynomialElasticNetCV,0.308417,0.283571,0.021272,0.112234
PolynomialLassoCV,0.301161,0.267077,0.018882,0.118067
PolynomialRidgeCV,0.255683,0.230196,0.017823,0.089409


## Result interpretation
### The top-2
**XGBRegressor is a clear winner**. The lowest score it got is over 0.64, mean and median scores are highest of all models, while score variance is low. It is clear that this algorithm reliably provides good results.

**GradientBoostingRegressor** is a close runner up, with similar characteristics, albeit somewhat less accurate and less consistent. This does not come as a surprise, since it uses a similar but less advanced algorithm to XGBoost. 

### Remaining results
The rest of the models have serious flaws. For example, **RandomForestRegressor**, despite having decent overall scores, has a higher score variance and got a score below 0.2 for one of the tracks. **ExtraTreesRegressor** is better in that regard, but still inferior to out top-2 models. 

The rest of the regressors perform significantly worse than the others, with versions of polynomial and linear regression having particularly low performance. There are some outlying values, even negative ones, in these models. Considering that XGBoost is a clear winner, I do not deem it necessary to look into this further at this point.

### To sum up
It appears that *boosting models*, particularly XGBRegressor and GradientBoostingRegressor, are the best. These are the models that will be optimized and tested further.

<br><br><br>


In [9]:
# Show how the best models perform on every circuit
relevant_scores = all_scores.loc[:, ["XGBRegressor", "GradientBoostingRegressor"]]
track_scores_df = pd.DataFrame(
    {
        "MeanScore": relevant_scores.mean(axis="columns"),
        "XGBRegressorScore": relevant_scores["XGBRegressor"],
        "GradientBoostingRegressorScore": relevant_scores["GradientBoostingRegressor"],
        "DataPointCount": [df.shape[0] for df in dfs.values()],
    }
)
track_scores_df.sort_values(by=["MeanScore"])

,MeanScore,XGBRegressorScore,GradientBoostingRegressorScore,DataPointCount
Hungaroring,0.617717,0.684105,0.551330,4951
Monte Carlo,0.621588,0.682995,0.560181,4244
Monza,0.643495,0.663191,0.623799,3898
Singapore,0.690026,0.741458,0.638594,3778
Catalunya,0.710296,0.724938,0.695654,5034
Baku,0.724269,0.714363,0.734176,2734
Sakhir,0.729789,0.746200,0.713378,4415
Silverstone,0.784176,0.800788,0.767565,2495
Miami,0.817181,0.827863,0.806499,2160
Yas Marina Circuit,0.822462,0.819814,0.825110,3307


## About scores by circuit
Scores clearly vary a lot depending on the circuit. It is important to note that both the characteristics of the circuit itself, as well as how much data we have on each circuit has a big effect. For some circuits we only have data from one session, which is an obvious limitation and could affect score in different ways. The score tends to be higher for circuits with less than 2000 data points, possibly because the data points only come from one or two sessions in those cases. This makes it very likely for weather to be roughly constant throughout the data relevant to them, skewing CV results in favour of the model.

### Point in favour of the results
Even in the worst case, XGBoost had a mean score of over 0.64, which means it accounted for 64% of target attribute variance. <br> 
The target attribute in this case the driver's lap time z-score within each session. Z-score in this case basically denotes how good the lap was compared to the other laps the same driver completed in the same session. This means that for the most unpredictable circuit, our model accounted for 64% of how pit stops and weather affect driver performance. For over 70% of the circuits, the model accounted for over 80% of those differences.

Therefore, it is clear to me that boosting models can produce decent-to-excellent results in general, even if some scores are exaggerated due to insufficient data size.